In [1]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\localt20\processedData'

batting_data = pl.read_csv(os.path.join(directory, 'batting.csv'))
bowling_data = pl.read_csv(os.path.join(directory, 'bowling.csv'))
fielding_data = pl.read_csv(os.path.join(directory, 'fielding.csv'))
batting_data

Player,Season,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,player_id
str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str
"""A Ahmadhel""","""2019/20""",3,2.0,16.0,100.0,8.0,"""Bulgaria""",8.0,16.0,2.0,100.0,"""55a5cffb"""
"""A Ahmadhel""","""2020""",1,1.0,8.0,100.0,8.0,"""Bulgaria""",8.0,24.0,3.0,100.0,"""55a5cffb"""
"""A Ahmadhel""","""2020/21""",2,1.0,2.0,28.57,2.0,"""Bulgaria""",6.5,26.0,4.0,76.19,"""55a5cffb"""
"""A Ahmadhel""","""2021""",3,3.0,5.0,38.46,1.66,"""Bulgaria""",4.43,31.0,7.0,66.76,"""55a5cffb"""
"""A Ahmadhel""","""2023""",2,0.0,0.0,0.0,0.0,"""Bulgaria""",4.43,31.0,7.0,53.41,"""55a5cffb"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zulqarnain Haider""","""2019""",4,1.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed"""
"""Zulqarnain Haider""","""2019/20""",1,0.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed"""
"""Zulqarnain Haider""","""2022""",6,3.0,8.0,66.66,4.0,"""Spain""",3.0,8.0,4.0,22.22,"""2d46e8ed"""


# Checking Nans

In [2]:
dfs = [batting_data, bowling_data, fielding_data]

for df in dfs:
    print(df.shape)
    # Identify numeric columns
    numeric_columns = df.select(pl.col(pl.Float32) | pl.col(pl.Float64)).columns

    # Check for NaN values in numeric columns
    nan_check = df.select([
        pl.col(column).is_nan().any().alias(f"{column}_has_nan") for column in numeric_columns
    ])

    print(nan_check)

(13060, 13)
shape: (0, 0)
┌┐
╞╡
└┘
(13060, 15)
shape: (0, 0)
┌┐
╞╡
└┘
(13060, 10)
shape: (0, 0)
┌┐
╞╡
└┘


In [3]:
player_data = batting_data.join(bowling_data, on=['player_id','Player',"Country","Season"], how='inner')
player_data = player_data.drop(['Mat_right','Inns_right'])
player_data

Player,Season,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,player_id,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ
str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A Ahmadhel""","""2019/20""",3,2.0,16.0,100.0,8.0,"""Bulgaria""",8.0,16.0,2.0,100.0,"""55a5cffb""",10.0,75.0,3.0,7.5,10.0,3.0,75.0,3.0,25.0
"""A Ahmadhel""","""2020""",1,1.0,8.0,100.0,8.0,"""Bulgaria""",8.0,24.0,3.0,100.0,"""55a5cffb""",2.0,22.0,1.0,11.0,12.0,4.0,97.0,4.0,24.25
"""A Ahmadhel""","""2020/21""",2,1.0,2.0,28.57,2.0,"""Bulgaria""",6.5,26.0,4.0,76.19,"""55a5cffb""",2.4,27.0,2.0,10.12,14.4,6.0,124.0,6.0,20.666667
"""A Ahmadhel""","""2021""",3,3.0,5.0,38.46,1.66,"""Bulgaria""",4.43,31.0,7.0,66.76,"""55a5cffb""",3.0,31.0,0.0,10.33,17.4,6.0,155.0,8.0,19.375
"""A Ahmadhel""","""2023""",2,0.0,0.0,0.0,0.0,"""Bulgaria""",4.43,31.0,7.0,53.41,"""55a5cffb""",0.0,0.0,0.0,0.0,17.4,6.0,155.0,8.0,19.375
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zulqarnain Haider""","""2019""",4,1.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",10.0,53.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25
"""Zulqarnain Haider""","""2019/20""",1,0.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",3.0,7.0,0.0,2.33,13.0,3.0,60.0,5.0,12.0
"""Zulqarnain Haider""","""2022""",6,3.0,8.0,66.66,4.0,"""Spain""",3.0,8.0,4.0,22.22,"""2d46e8ed""",15.0,91.0,5.0,6.06,28.0,8.0,151.0,11.0,13.727273


In [4]:
player_data

Player,Season,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,player_id,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ
str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A Ahmadhel""","""2019/20""",3,2.0,16.0,100.0,8.0,"""Bulgaria""",8.0,16.0,2.0,100.0,"""55a5cffb""",10.0,75.0,3.0,7.5,10.0,3.0,75.0,3.0,25.0
"""A Ahmadhel""","""2020""",1,1.0,8.0,100.0,8.0,"""Bulgaria""",8.0,24.0,3.0,100.0,"""55a5cffb""",2.0,22.0,1.0,11.0,12.0,4.0,97.0,4.0,24.25
"""A Ahmadhel""","""2020/21""",2,1.0,2.0,28.57,2.0,"""Bulgaria""",6.5,26.0,4.0,76.19,"""55a5cffb""",2.4,27.0,2.0,10.12,14.4,6.0,124.0,6.0,20.666667
"""A Ahmadhel""","""2021""",3,3.0,5.0,38.46,1.66,"""Bulgaria""",4.43,31.0,7.0,66.76,"""55a5cffb""",3.0,31.0,0.0,10.33,17.4,6.0,155.0,8.0,19.375
"""A Ahmadhel""","""2023""",2,0.0,0.0,0.0,0.0,"""Bulgaria""",4.43,31.0,7.0,53.41,"""55a5cffb""",0.0,0.0,0.0,0.0,17.4,6.0,155.0,8.0,19.375
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zulqarnain Haider""","""2019""",4,1.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",10.0,53.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25
"""Zulqarnain Haider""","""2019/20""",1,0.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",3.0,7.0,0.0,2.33,13.0,3.0,60.0,5.0,12.0
"""Zulqarnain Haider""","""2022""",6,3.0,8.0,66.66,4.0,"""Spain""",3.0,8.0,4.0,22.22,"""2d46e8ed""",15.0,91.0,5.0,6.06,28.0,8.0,151.0,11.0,13.727273


In [5]:
player_data = player_data.join(fielding_data, on=['player_id','Player',"Country","Season"], how='inner')
player_data = player_data.drop(['Mat_right','Inns_right'])
player_data


Player,Season,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,player_id,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A Ahmadhel""","""2019/20""",3,2.0,16.0,100.0,8.0,"""Bulgaria""",8.0,16.0,2.0,100.0,"""55a5cffb""",10.0,75.0,3.0,7.5,10.0,3.0,75.0,3.0,25.0,0.0,0.0,0.0,0.0
"""A Ahmadhel""","""2020""",1,1.0,8.0,100.0,8.0,"""Bulgaria""",8.0,24.0,3.0,100.0,"""55a5cffb""",2.0,22.0,1.0,11.0,12.0,4.0,97.0,4.0,24.25,0.0,0.0,0.0,0.0
"""A Ahmadhel""","""2020/21""",2,1.0,2.0,28.57,2.0,"""Bulgaria""",6.5,26.0,4.0,76.19,"""55a5cffb""",2.4,27.0,2.0,10.12,14.4,6.0,124.0,6.0,20.666667,0.0,0.0,0.0,0.0
"""A Ahmadhel""","""2021""",3,3.0,5.0,38.46,1.66,"""Bulgaria""",4.43,31.0,7.0,66.76,"""55a5cffb""",3.0,31.0,0.0,10.33,17.4,6.0,155.0,8.0,19.375,0.0,0.0,0.0,0.0
"""A Ahmadhel""","""2023""",2,0.0,0.0,0.0,0.0,"""Bulgaria""",4.43,31.0,7.0,53.41,"""55a5cffb""",0.0,0.0,0.0,0.0,17.4,6.0,155.0,8.0,19.375,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zulqarnain Haider""","""2019""",4,1.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",10.0,53.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25,1.0,1.0,0.0,0.25
"""Zulqarnain Haider""","""2019/20""",1,0.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed""",3.0,7.0,0.0,2.33,13.0,3.0,60.0,5.0,12.0,1.0,1.0,0.0,0.2
"""Zulqarnain Haider""","""2022""",6,3.0,8.0,66.66,4.0,"""Spain""",3.0,8.0,4.0,22.22,"""2d46e8ed""",15.0,91.0,5.0,6.06,28.0,8.0,151.0,11.0,13.727273,1.0,1.0,0.0,0.090909


In [6]:
for i in player_data.columns:
    if player_data[i].dtype != pl.String:
        if player_data[i].is_nan().sum()>0:
            print(i,player_data[i].is_nan().sum())

In [7]:
player_data.write_csv(r'D:\github\localt20\processedData\playerStats.csv')